In [1]:
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import math
from math import sqrt, floor
from heapq import heappop, heappush
from datetime import datetime
%matplotlib inline

import sys
sys.path.append('anya')

from grid import Map
from anya import anya

In [2]:
from theta.src.utils import *

In [3]:
class NewStats(Stats):
    def __init__(self):
        self.difficulty = 0
        self.expansions = 0 # algorithm must set this value
        self.runtime = 0 # algorithm must set this value
        self.way_length = 0 # algorithm must set this value
        self.suboptimal = 0
        self.max_tree_size = 0 # algorithm must set this value

    def read_from_string(self, data):
        delimiter = ","
        diff, expansions, runtime, way_length, suboptimal, tree_size = data.split(delimiter)
        self.difficulty = int(diff)
        self.expansions = int(expansions)
        self.runtime = float(runtime)
        self.way_length = float(way_length)
        self.suboptimal = int(suboptimal)
        self.max_tree_size = int(max_tree_size)
        
        
    def __repr__(self):
        delimiter = ","
        return str(self.difficulty) + delimiter + str(self.expansions) + delimiter + str(self.runtime) + delimiter + str(self.way_length) +\
        delimiter + str(self.suboptimal)  + delimiter + str(self.max_tree_size)
    
    def header(self):
        delimiter = ","
        return "difficulty" + delimiter + "expansions" + delimiter + "runtime" + delimiter + "way_length" +\
        delimiter + "suboptimal" + delimiter + "max_tree_size"
  

In [4]:
def compute_stats(stats, node, optimal):
    if not node is None:
        stats.suboptimal = round(stats.way_length / optimal, 6)
        stats.runtime = round(stats.runtime.total_seconds(), 6)
    return stats

In [5]:
class TestCase:
    def __init__(self, difficulty, start, goal, optimal):
        self.difficulty = difficulty
        self.start = start
        self.goal = goal
        self.optimal = optimal

In [6]:
def read_map_from_file(path):
    with open(path, "r") as f:
        map_type = f.readline()
        height = f.readline().split()[1]
        width = f.readline().split()[1]
        f.readline()
        # now we read the map
        map_canvas = "\n".join(f.readlines())
        return map_canvas, int(width), int(height)

In [7]:
def read_scenes_from_file(path):
    tests = []
    with open(path, "r") as f:
        version = f.readline()
        for data in f.readlines():
            difficulty, _,_,_, start_y, start_x, goal_y, goal_x, optimal = data.split()
            test = TestCase(int(difficulty), (int(start_x), int(start_y)), (int(goal_x), int(goal_y)), float(optimal))
            tests.append(test)
    return tests

In [8]:
class TestSet:
    def __init__(self, name, path, output):
        self.map = Map()
        self.tests = []
        self.max_diff = 0
        self.name = name
        
        self.read_from_file(path)
        self.input_path = path
        self.output_path = output
    
    def read_from_file(self, path):
        map_data = read_map_from_file(path + ".map")
        self.map.read_from_string(*map_data)
        self.tests = read_scenes_from_file(path + ".map.scen")
        self.max_diff = max([item.difficulty for item in self.tests])
        
    def choose_tests(self, number):
        tests = []
        i = 0
        for diff in range(number):
            target = diff * self.max_diff  // number
            while self.tests[i].difficulty != target:
                i += 1
            tests.append(self.tests[i])
            i += 1
        return tests

In [9]:
sets = []
setSC = TestSet("starcraft", "analysis/data/starcraft/NovaStation", "analysis/results/starcraft/NovaStation")
sets.append(setSC)
setCity = TestSet("paris", "analysis/data/paris/Paris_1_256", "analysis/results/paris/Paris_1_256")
sets.append(setCity)
setMaze = TestSet("maze", "analysis/data/maze/maze512-32-7", "analysis/results/maze/maze512-32-7")
sets.append(setMaze)
# setRoom = TestSet("room", "analysis/data/room/16room_005", "analysis/results/room/16room_005")
# sets.append(setRoom)
setRand = TestSet("random", "analysis/data/random/random512-10-5", "analysis/results/random/random512-10-5")
sets.append(setRand)

In [10]:
class SearchCase:
    def __init__(self, h_i, w_i, p_i, k_i):
        self.h_i = h_i
        self.w_i = w_i
        self.p_i = p_i
        self.k_i = k_i
        
    def _repr__(self):
        return names[self.h_i] + ".w=" + str(ws[self.w_i]) + ".p=" + str(ps[self.p_i]) + ".k=" + str(ks[self.k_i])

In [11]:
def check(testSet, test_number):
    res = []
    tests = testSet.choose_tests(test_number)
    
    for test in tests:
        print(testSet.map.height, testSet.map.width, test.start, test.goal)
        length, path = anya(testSet.map, test.start, test.goal)
        res.append(length)
        for i in range(1, len(path)):
            assert testSet.map.visible(path[i-1][0], path[i-1][1], path[i][0], path[i][1])

    return res
    

In [12]:
check(setSC, 2)

512 512 (450, 140) (451, 142)
512 512 (441, 423) (338, 90)


AssertionError: 